# Controller Manager

參考官方資料: https://kubernetes.io/docs/concepts/architecture/controller/

<img src='./img/12.png'>

# Controllers

# System Pods

In [ ]:
kubectl get all 

In [ ]:
kubectl get all -A

In [ ]:
kubectl get daemonset --namespace kube-system

In [ ]:
kubectl get deployment coredns --namespace kube-system

# Labels

### Add labels

##### 方法一: Declaratively in a Manifest in YAML

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx-pod-1
  labels:
    app: v1
    tier: PROD
spec:
  containers:
  - name: nginx
    image: nginx
---
apiVersion: v1
kind: Pod
metadata:
  name: nginx-pod-2
  labels:
    app: v1
    tier: ACC
spec:
  containers:
  - name: nginx
    image: nginx
---
apiVersion: v1
kind: Pod
metadata:
  name: nginx-pod-3
  labels:
    app: v1
    tier: TEST
spec:
  containers:
  - name: nginx
    image: nginx

##### 方法二:  Imperatively with kubectl

In [ ]:
kubectl label pod nginx-pod-1 tier=PROD app=v1
kubectl label pod nginx-pod-2 tier=ACC app=v1
kubectl label pod nginx-pod-3 tier=TEST app=v1

### Update labels

In [ ]:
kubectl label pod nginx-pod-1 tier=ACC app=v1 --overwrite

### Delete labels

In [ ]:
kubectl label pod nginx-pod-1 tier-

### Querying using labels and selectors

In [ ]:
kubectl get pods --show-labels
kubectl get pods --selector tier=PROD

---

In [ ]:
kubectl get pods -l 'tier in (PROD, TEST)'

---

In [ ]:
kubectl get pods -l 'tier notin (PROD, TEST)'

### How kubernetes uses labels

# Deployments

參考官方資料: https://kubernetes.io/docs/concepts/workloads/controllers/deployment/

### 範例01

In [ ]:
kubectl config set-context --current --namespace kube-system
kubectl config get-contexts
kubectl get deployments.apps

<img src='./img/13.png'>

In [ ]:
kubectl describe deployments.apps coredns

<img src='./img/14.png'>

---

In [ ]:
kubectl get replicasets.apps coredns-64897985d
kubectl describe replicasets.apps coredns-64897985d

<img src='./img/15.png'>

In [ ]:
kubectl get pods --show-labels 

<img src='./img/22.png'>

In [ ]:
kubectl describe pods coredns-64897985d-lgvxx

<img src='./img/17.png'>

# Creating a Deployment

### 方法一: Imperatively

In [ ]:
kubectl create deployment web --image=nginx:1.14.2
kubectl get deployments.apps
kubectl get replicasets.apps
kubectl get pod --show-labels

In [ ]:
kubectl scale deployment web --replicas=5
kubectl describe deployments.apps web

<img src='./img/18.png'>

In [ ]:
kubectl describe replicasets.apps web-656c4c457f

<img src='./img/19.png'>

In [ ]:
kubectl describe replicasets.apps web-656c4c457f

### 方法二: Declaratively

In [ ]:
apiVersion: apps/v1 # for versions before 1.9.0 use apps/v1beta2
kind: Deployment
metadata:
  labels:
    app: web
  name: web
spec:
  replicas: 2 # tells deployment to run 2 pods matching the template
  selector:
    matchLabels:
      app: web
  template: # create pods using pod definition in this template
    metadata:
      labels:
        app: web
    spec:
      containers:
      - image: nginx:1.14.2
        name: nginx
        ports:
        - containerPort: 80

In [ ]:
kubectl apply -f nginx_deployment.yml

# Getting deployments

In [ ]:
kubectl get deployments

In [ ]:
kubectl get deployments -o wide 

In [ ]:
kubectl get replicasets

In [ ]:
kubectl get pods -o wide

In [ ]:
kubectl get pods -l app=web

In [ ]:
kubectl describe deployment web

# Pod failures

In [ ]:
kubectl get pod -l app=web

In [ ]:
kubectl describe pod podName

In [ ]:
kubectl delete pod podName

In [ ]:
kubectl get pods -l app=web

### 範例01

In [ ]:
kubectl create deployment web --image=nginx
kubectl get deployments.apps
kubectl scale deployment web --replicas=3

In [ ]:
kubectl get pods -l app=web
kubectl delete pod podName

In [ ]:
kubectl get pods --show-labels
kubectl label pod podName app=test --overwrite

In [ ]:
kubectl get pods --show-labels
kubectl label pod  podName app=web --overwrite

# Node failures

### 範例01

# Deployment update strategy: RollingUpdate

In [ ]:
kubectl create deployment web --image=nginx:1.14 --replicas=3

In [ ]:
kubectl describe deployment web

<img src='./img/20.png'>

# Deployment rollout and restart

### rollout 的操作

In [ ]:
kubectl rollout history deployment web

In [ ]:
kubectl rollout history deployment web --revision 1

In [ ]:
kubectl rollout undo deployment web --to-revision 1

In [ ]:
kubectl explain deployment.spec

### restart 的操作

In [ ]:
kubectl rollout restart deployment web

# Deployment update strategy: Recreate

### 範例

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  creationTimestamp: null
  labels:
    app: web
  name: web
spec:
  replicas: 3
  selector:
    matchLabels:
      app: web
  strategy:
    type: Recreate 
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: web
    spec:
      containers:
      - image: nginx:1.14.2
        name: nginx
        resources: {}
status: {}

# Updating the deployment

### 方法一: 使用 ymal 更新

In [ ]:
apiVersion: apps/v1 # for versions before 1.9.0 use apps/v1beta2
kind: Deployment
metadata:
  labels:
    app: web
  name: web
spec:
  replicas: 2 # tells deployment to run 2 pods matching the template
  selector:
    matchLabels:
      app: web
  template: # create pods using pod definition in this template
    metadata:
      labels:
        app: web
    spec:
      containers:
      - image: nginx:1.20.0
        name: nginx
        ports:
        - containerPort: 80

In [ ]:
kubectl apply -f nginx_deployment_update.yaml

In [ ]:
kubectl describe pod podName

---

In [ ]:
apiVersion: apps/v1 
kind: Deployment
metadata:
  labels:
    app: web
  name: web
spec:
  replicas: 4
  selector:
    matchLabels:
      app: web
  template: 
    metadata:
      labels:
        app: web
    spec:
      containers:
      - image: nginx:1.20.0
        name: nginx
        ports:
        - containerPort: 80

In [ ]:
kubectl apply -f nginx_deployment_scale.yaml

In [ ]:
kubectl get pods -l app=web
kubecyl get deployments 

### 方法二: 使用 edit 命令

In [ ]:
kubectl edit deployment deploymentName

In [ ]:
kubectl get deployment deploymentName

### 方法三: 使用 scale 指令改 replicas 個數

In [ ]:
kubectl scale deployment deploymentName --replicas=4

In [ ]:
kubectl get deployment deploymentName

### 方法四: 使用 set 命令改 image 版本

In [ ]:
kubectl set image deployment/deploymentName nginx=nginx:1.9.1

In [ ]:
kubectl get deployment deploymentName -o wide

# Deleting a Deployment

In [ ]:
kubectl delete deployment nginx-deployment

# DaemonSet

參考官方文件: https://kubernetes.io/docs/concepts/workloads/controllers/daemonset/

In [ ]:
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: hello-ds
spec:
  selector:
    matchLabels:
      app: hello-world
  template:
    metadata:
      labels:
        app: hello-world
    spec:
      containers:
      - name: hello-world
        image: nginx:1.14

In [ ]:
kubectl apply -f daemonset.yml

In [ ]:
kubectl get daemonsets

In [ ]:
kubectl get daemonsets -o wide

In [ ]:
kubectl get pods -o wide

### 範例

In [ ]:
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-elasticsearch
  namespace: kube-system
  labels:
    k8s-app: fluentd-logging
spec:
  selector:
    matchLabels:
      name: fluentd-elasticsearch
  template:
    metadata:
      labels:
        name: fluentd-elasticsearch
    spec:
      tolerations:
      # these tolerations are to have the daemonset runnable on control plane nodes
      # remove them if your control plane nodes should not run pods
      - key: node-role.kubernetes.io/control-plane
        operator: Exists
        effect: NoSchedule
      - key: node-role.kubernetes.io/master
        operator: Exists
        effect: NoSchedule
      containers:
      - name: fluentd-elasticsearch
        image: quay.io/fluentd_elasticsearch/fluentd:v2.5.2
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
        volumeMounts:
        - name: varlog
          mountPath: /var/log
      terminationGracePeriodSeconds: 30
      volumes:
      - name: varlog
        hostPath:
          path: /var/log

### Update Strategy

### 方法一: OnDelete

### 方法二: RollingUpdate(預設)

# Job

### Job Spec 格式

### 範例

##### 方法一: 使用指令

In [ ]:
kubectl create job my-job --image=busybox -- sh -c "sleep 50"

##### 方法二: 使用 yaml 檔

In [ ]:
apiVersion: batch/v1
kind: Job
metadata:
  name: my-job
spec:
  template:
    spec:
      containers:
      - name: my-job
        image: busybox
        command: ["sh",  "-c", "sleep 50"]
      restartPolicy: Never

In [ ]:
kubectl create -f job.yaml

In [ ]:
kubectl get pod -o wide

In [ ]:
kubectl get job

# CronJob

### 什麼是 Cronjob

<img src='./img/23.png'>

### Crontab on MacOS

In [ ]:
$ crontab -e

In [ ]:
*/1 * * * * echo "Hi, current time is $(date)" >> ~/cronjob.log

<img src='./img/24.png'>

<img src='./img/25.png'>

### 在 minikube 上實現排程服務

In [ ]:
apiVersion: batch/v1beta1
kind: CronJob
metadata:
  name: hello
spec:
  schedule: "*/1 * * * *"
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: hello
            image: busybox
            args:
            - /bin/sh
            - -c
            - echo "Hi, current time is $(date)"
          restartPolicy: OnFailure

In [ ]:
kubectl create -f ./my-cronjob.yaml

In [ ]:
kubectl get cronjobs

In [ ]:
kubectl get jobs --watch

### 查看 Logs

In [ ]:
kubectl get pods -a --show-all=true -o wide --show-labels=true

<img src='./img/26.png'>

<img src='./img/27.png'>

<img src='./img/28.png'>